In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, f1_score
from tensorflow import keras
from tensorflow.keras import layers
import ipywidgets as widgets
from IPython.display import display
from joblib import load
from tensorflow.keras.models import save_model
from joblib import dump
from tensorflow.keras.models import load_model

In [14]:
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

# Path to the CSV file
csv_path = '/content/drive/My Drive/366 PROJECT/DATASET/houserentdhaka.csv'

# Import necessary libraries
import pandas as pd

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(csv_path)

# Display the DataFrame
print(df)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
       Unnamed: 0                                           Location  \
0               0                    Block H, Bashundhara R-A, Dhaka   
1               1                           Farmgate, Tejgaon, Dhaka   
2               2  Block B, Nobodoy Housing Society, Mohammadpur,...   
3               3                          Gulshan 1, Gulshan, Dhaka   
4               4                                   Baridhara, Dhaka   
...           ...                                                ...   
28795        4795                           Section 2, Mirpur, Dhaka   
28796        4796                    Block F, Bashundhara R-A, Dhaka   
28797        4797         RK Mission Road, Gopibag, Motijheel, Dhaka   
28798        4798                Ullan, West Rampura, Rampura, Dhaka   
28799        4799                               Vatara, Badda, Dhaka   

      

In [15]:
# Preprocess the data
le = LabelEncoder()
df['Location'] = le.fit_transform(df['Location'])
df['Area'] = df['Area'].str.replace(' sqft', '').str.replace(',', '').astype(float)

# Convert the "Price" column to numerical values
df['Price'] = df['Price'].replace({' Thousand': 'e3', ' Lakh': 'e5'}, regex=True).map(pd.eval).astype(float)

# Split the data into features (X) and target variable (y)
X = df.drop('Price', axis=1)
y = df['Price']

# Standardize numerical features
scaler = StandardScaler()
X[['Area', 'Bed', 'Bath']] = scaler.fit_transform(X[['Area', 'Bed', 'Bath']])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Build the neural network model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=100, verbose=1)


Epoch 1/100
720/720 [==============================] - 5s 3ms/step - loss: 601313536.0000
Epoch 2/100
720/720 [==============================] - 2s 3ms/step - loss: 553184384.0000
Epoch 3/100
720/720 [==============================] - 2s 2ms/step - loss: 549518720.0000
Epoch 4/100
720/720 [==============================] - 2s 2ms/step - loss: 545201792.0000
Epoch 5/100
720/720 [==============================] - 2s 2ms/step - loss: 537585536.0000
Epoch 6/100
720/720 [==============================] - 2s 2ms/step - loss: 521832416.0000
Epoch 7/100
720/720 [==============================] - 2s 2ms/step - loss: 485874208.0000
Epoch 8/100
720/720 [==============================] - 2s 2ms/step - loss: 418816352.0000
Epoch 9/100
720/720 [==============================] - 2s 3ms/step - loss: 341912896.0000
Epoch 10/100
720/720 [==============================] - 2s 3ms/step - loss: 286163712.0000
Epoch 11/100
720/720 [==============================] - 2s 2ms/step - loss: 261875216.0000
Epoch 12

In [ ]:
# Path to save the model
model_save_path = '/content/drive/My Drive/366 PROJECT/MODELS/keras_neural_network_model'

# Save the trained Keras Neural Network model
model.save(model_save_path)

In [ ]:
# Build the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the Random Forest model
rf_model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [ ]:
# Path to save the model
model_save_path = '/content/drive/My Drive/366 PROJECT/MODELS/random_forest_model.joblib'

# Save the trained Random Forest model
dump(rf_model, model_save_path)

['/content/drive/My Drive/366 PROJECT/MODELS/random_forest_model.joblib']

In [16]:
# Path to load the Keras Neural Network model
model_load_path_keras = '/content/drive/My Drive/366 PROJECT/MODELS/keras_neural_network_model'

# Load the trained Keras Neural Network model
loaded_keras_model = load_model(model_load_path_keras)

# Path to load the Random Forest model
model_load_path_rf = '/content/drive/My Drive/366 PROJECT/MODELS/random_forest_model.joblib'

# Load the trained Random Forest model
loaded_rf_model = load(model_load_path_rf)

In [19]:
# Evaluate Neural Network model
y_pred_nn = loaded_keras_model.predict(X_test)
mse_nn = mean_squared_error(y_test, y_pred_nn)
r2_nn = r2_score(y_test, y_pred_nn)
f1_nn = f1_score(y_test, y_pred_nn.round(), average='weighted')

print('Metrics for Neural Network:')
print(f'Mean Squared Error: {mse_nn}')
print(f'R-squared Value: {r2_nn}')
print(f'F1 Score: {f1_nn}')

# Evaluate Random Forest model
y_pred_rf = loaded_rf_model.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf.round(), average='weighted')

print('\nMetrics for Random Forest:')
print(f'Mean Squared Error: {mse_rf}')
print(f'R-squared Value: {r2_rf}')
print(f'F1 Score: {f1_rf}')


180/180 [==============================] - 1s 5ms/step
Metrics for Neural Network:
Mean Squared Error: 85970744.4849648
R-squared Value: 0.7951438540456544
F1 Score: 0.0003452151573538857

Metrics for Random Forest:
Mean Squared Error: 43979480.019418925
R-squared Value: 0.8952031085478158
F1 Score: 0.04093408145885082


In [17]:
# Create dropdown for location
location_dropdown = widgets.Dropdown(
    options=le.classes_,
    value=le.classes_[0],
    description='Location:',
)

# Create text inputs for other features
area_text = widgets.Text(description='Area (sqft):', value='1000')
bed_text = widgets.Text(description='Bedrooms:', value='3')
bath_text = widgets.Text(description='Bathrooms:', value='2')

# Create buttons for predictions
predict_button_nn = widgets.Button(description='Predict Price (Neural Network)')
predict_button_rf = widgets.Button(description='Predict Price (Random Forest)')

# Function to be called when the buttons are clicked for Neural Network and Random Forest
def predict_price(button):
    user_location = location_dropdown.value
    user_area = float(area_text.value.replace(',', ''))
    user_bed = int(bed_text.value)

    # Check if 'Bathrooms' input is not empty
    user_bath_text = bath_text.value
    user_bath = int(user_bath_text) if user_bath_text else 0

    # Preprocess user input
    user_location_encoded = le.transform([user_location])[0]
    user_data = pd.DataFrame({'Unnamed: 0': [0], 'Location': [user_location_encoded], 'Area': [user_area], 'Bed': [user_bed], 'Bath': [user_bath]})
    user_data[['Area', 'Bed', 'Bath']] = scaler.transform(user_data[['Area', 'Bed', 'Bath']])

    if button.description == 'Predict Price (Neural Network)':
        # Predict the price using Neural Network
        predicted_price = loaded_keras_model.predict(user_data)
        print(f'Predicted Price (Neural Network): {predicted_price[0][0]}')
    elif button.description == 'Predict Price (Random Forest)':
        # Predict the price using Random Forest
        predicted_price = loaded_rf_model.predict(user_data)
        print(f'Predicted Price (Random Forest): {predicted_price[0]}')

# Attach the function to the button click events
predict_button_nn.on_click(predict_price)
predict_button_rf.on_click(predict_price)

# Display the widgets
display(location_dropdown, area_text, bed_text, bath_text, predict_button_nn, predict_button_rf)

Dropdown(description='Location:', options=('1st Colony, Mirpur, Dhaka', '22 Bari, West Kazipara, Mirpur, Dhaka…

Text(value='1000', description='Area (sqft):')

Text(value='3', description='Bedrooms:')

Text(value='2', description='Bathrooms:')

Button(description='Predict Price (Neural Network)', style=ButtonStyle())

Button(description='Predict Price (Random Forest)', style=ButtonStyle())

1/1 [==============================] - 0s 47ms/step
Predicted Price (Neural Network): 14064.869140625
Predicted Price (Random Forest): 17425.0
1/1 [==============================] - 0s 19ms/step
Predicted Price (Neural Network): 107773.359375
Predicted Price (Random Forest): 79750.0
1/1 [==============================] - 0s 17ms/step
Predicted Price (Neural Network): 173159.109375
Predicted Price (Random Forest): 114750.0
